In [116]:
SELECT 2 + 2;

(1 row affected)

Total execution time: 00:00:00.081

(No column name)
4


In [2]:
IF NOT EXISTS (
        SELECT *
        FROM sys.schemas
        WHERE name = N'BH'
    )
    EXEC('CREATE SCHEMA [BH]');
GO

Commands completed successfully.

Total execution time: 00:00:00.104

In [117]:
IF NOT EXISTS (
    SELECT * FROM sys.objects
    WHERE object_id = OBJECT_ID(N'BH.Customer') 
)

BEGIN
CREATE TABLE BH.Customer (
    Id int IDENTITY(1, 1) PRIMARY KEY,
    FirstName VARCHAR(255),
    SecondName VARCHAR(255),
    LastName VARCHAR(255),
    TerritoryName VARCHAR(255)
);
END

Commands completed successfully.

Total execution time: 00:00:00.063

In [118]:
INSERT INTO BH.Customer (
    FirstName, SecondName, LastName, TerritoryName
) VALUES
    ('Bohdan', 'Hlovatskyui', 'M', 'Ukraine'),
    ('HM', 'HH', 'M', 'Australia'),
    ('HHH', 'H', 'H', 'Canada');

(3 rows affected)

Total execution time: 00:00:00.058

In [119]:
SELECT * FROM BH.Customer;

(3 rows affected)

Total execution time: 00:00:00.055

Id,FirstName,SecondName,LastName,TerritoryName
1,Bohdan,Hlovatskyui,M,Ukraine
2,HM,HH,M,Australia
3,HHH,H,H,Canada


In [120]:
INSERT INTO BH.Customer (FirstName, LastName, TerritoryName)
(
    SELECT p.FirstName, p.LastName, t.Name
    FROM Sales.Customer AS s
    LEFT JOIN Person.Person AS p
    ON s.CustomerID = p.BusinessEntityID
    LEFT JOIN Sales.SalesTerritory AS t
    ON s.TerritoryID = t.TerritoryID
    LEFT JOIN Sales.SalesOrderHeader AS o 
    ON s.CustomerID = o.CustomerID
);

In [ ]:
SELECT p.FirstName, p.LastName, t.Name
FROM Sales.Customer AS s
LEFT JOIN Person.Person AS p
ON s.CustomerID = p.BusinessEntityID
LEFT JOIN Sales.SalesTerritory AS t
ON s.TerritoryID = t.TerritoryID
LEFT JOIN Sales.SalesOrderHeader AS o 
ON s.CustomerID = o.CustomerID;

In [123]:
IF NOT EXISTS (
    SELECT * FROM sys.objects
    WHERE object_id = OBJECT_ID(N'BH.Orders') 
)
BEGIN
CREATE TABLE BH.Orders (
    Id int IDENTITY(1, 1) PRIMARY KEY,
    CustomerId int NOT NULL,
    OrderDate date,
    Quantity int,

    CONSTRAINT FK_Orders_CustomerId FOREIGN KEY(CustomerId) REFERENCES BH.Customer(Id),
    CONSTRAINT CK_Orders_Qunatity CHECK (Quantity >= 1)
);
END

Commands completed successfully.

Total execution time: 00:00:00.065

In [127]:
INSERT INTO BH.Orders (CustomerId, OrderDate, Quantity) VALUES
(3, CAST('07-09-2006' AS DATE), 7)
GO 2

Beginning execution loop

(1 row affected)

(1 row affected)

Batch execution completed 2 times...

Total execution time: 00:00:00.111

In [128]:
ALTER TABLE BH.Orders
 ADD CONSTRAINT BH_Orders_OrderDate DEFAULT GETDATE() FOR OrderDate

Commands completed successfully.

Total execution time: 00:00:00.059

In [129]:
SELECT * FROM BH.Orders

(6 rows affected)

Total execution time: 00:00:00.064

Id,CustomerId,OrderDate,Quantity
3,1,2004-07-05,5
4,1,2004-07-05,5
5,2,2008-07-05,3
6,2,2008-07-05,3
7,3,2006-07-09,7
8,3,2006-07-09,7


In [130]:
ALTER TABLE BH.Orders
ADD Comments varchar(255) DEFAULT '<None>'

Commands completed successfully.

Total execution time: 00:00:00.060

In [131]:
SELECT * FROM BH.Orders

(6 rows affected)

Total execution time: 00:00:00.073

Id,CustomerId,OrderDate,Quantity,Comments
3,1,2004-07-05,5,NULL
4,1,2004-07-05,5,NULL
5,2,2008-07-05,3,NULL
6,2,2008-07-05,3,NULL
7,3,2006-07-09,7,NULL
8,3,2006-07-09,7,NULL


In [132]:
DELETE FROM BH.Orders
WHERE OrderDate < CAST('AUGUST 2006' AS DATE)

(4 rows affected)

Total execution time: 00:00:00.080

In [133]:
DELETE BH.Orders FROM BH.Orders AS t1
JOIN BH.Customer AS t2 ON t1.CustomerId = t2.Id
WHERE t2.TerritoryName = 'Canada'

(0 rows affected)

Total execution time: 00:00:00.078

Try delete from Customer table one customer who has  some orders (see what happens)


In [134]:
BEGIN TRANSACTION;
BEGIN TRY   
    DELETE BH.Customer FROM BH.Customer AS c
    -- inner join as we are interested only in those that
    -- made the order
    INNER JOIN BH.Orders AS o ON c.Id = o.CustomerId
    COMMIT TRANSACTION;
END TRY
BEGIN CATCH
  ROLLBACK TRANSACTION;
END CATCH

(0 rows affected)

Total execution time: 00:00:00.076

Update all orders in Orders table that were placed by Australia customers and set Quantity = 5 for these orders

In [64]:
UPDATE BH.Orders
SET Quantity = 5
FROM BH.Orders AS o
INNER JOIN BH.Customer AS c
ON o.CustomerId = c.Id 
WHERE c.TerritoryName = 'Australia';

(0 rows affected)

Total execution time: 00:00:00.131

In [65]:
UPDATE BH.Orders
SET Quantity = Quantity * 1.1 
FROM BH.Orders AS o
INNER JOIN BH.Customer AS c
ON o.CustomerId = c.Id 
WHERE c.TerritoryName = 'Canada';

(0 rows affected)

Total execution time: 00:00:00.055

In [70]:
-- note that here we need o.Id
SELECT o.Id, CustomerId, OrderDate, Quantity INTO BH.Orders_Sub
FROM BH.Orders AS o
JOIN BH.Customer AS c ON o.CustomerId = c.Id
WHERE FirstName LIKE 'N%' or LastName in ('Scott')

(0 rows affected)

Total execution time: 00:00:00.642

In [72]:
UPDATE BH.Orders_Sub
SET OrderDate = CAST('2012-12-12' AS DATE)
FROM BH.Orders_Sub WHERE Quantity=5

(0 rows affected)

Total execution time: 00:00:00.109

In [74]:
-- this one is the fastest, as delete processes row-by-row and
-- drop deletes the table as well
TRUNCATE TABLE BH.Orders_Sub;

Commands completed successfully.

Total execution time: 00:00:00.109

In [135]:
DROP TABLE IF EXISTS BH.Orders, BH.Orders_Sub, BH.Customer;

Commands completed successfully.

Total execution time: 00:00:00.073